In [ ]:
import pandas as pd
import numpy as np
import os
import joblib
import time
import lightgbm as lgb

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import PredefinedSplit 

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline

In [4]:
TEST_SIZE = 0.20
RANDOM_STATE = 42
CV_FOLDS = 3
N_JOBS = 4

In [ ]:
base_path = os.path.dirname(os.getcwd())
data_path = os.path.join(base_path, "data", "processed", "diplomacy", "diplomacy_processed.parquet")
data = pd.read_parquet(data_path)

In [8]:
print(data.columns)

Index(['game_id', 'speaker', 'receiver', 'message_text', 'sender_intention',
       'game_score', 'game_score_delta', 'year', 'season', 'original_fold',
       'target', 'cleaned_text', 'message_length'],
      dtype='object')


In [12]:
X = data.drop("target", axis=1)
y = data["target"]